In [40]:
import os
import copy
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets 
import torchvision.transforms as transforms
import pandas as pd
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts,ExponentialLR
from sklearn.model_selection import train_test_split
from PIL import Image
import cv2
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

In [41]:
"""
为每个标签创建一个唯一的数字编号（从0开始），
并将这个数字编号作为新的列添加到DataFrame中，最后保存为新的CSV文件。
"""
train = pd.read_csv("../dataset/train.csv")
labels = list(train['label'])  # 提取标签列并转换为列表
labels_unique = list(set(labels))   # set去除重复标签，list转换为唯一标签列表

# 创建数字标签列表
label_nums = []
for i in range(len(labels)):
    # 将文本标签转换为数字编号：
    # 当前标签在唯一标签列表中的索引作为其数字标签
    label_nums.append(labels_unique.index(labels[i]))
    
train['number'] = label_nums
train.to_csv("../dataset/train_num_label.csv", index=0)  # 记录对应关系

In [42]:
# 读取测试集
test_df = pd.read_csv("../dataset/test.csv")  
# 将数据集 train 分割成训练集和验证集
# stratify：按数字标签进行分层抽样，确保训练集和验证集中各类别的比例相同
train_data, eval_data = train_test_split(train, test_size=0.2, stratify=train['number'])  

In [43]:
class Leaf_Dataset(Dataset):
    '''
    树叶数据集的训练集 自定义Dataset
    '''
    def __init__(self, train_csv, transform=None, test=False):
        '''
        train_csv : 传入记录图像路径及其标号的csv文件
        transform : 对图像进行的变换
        test : 是否为测试集模式
        '''
        super().__init__()
        self.train_csv = train_csv
        self.image_path = list(self.train_csv['image'])  # 提取所有图像路径
        self.transform = transform  
        self.test = test
        # 如果不是测试集，加载标签信息
        if not self.test:
            self.label_nums = list(self.train_csv['number'])  # 提取数字标签列表

    def __getitem__(self, idx):
        '''
        获取单个样本
        idx : 样本索引
        return : image, label
        '''
        # 读取图像
        image = cv2.imread(os.path.join('../dataset', self.image_path[idx]))
        # 转换颜色空间 BGR→RGB (OpenCV默认BGR，PyTorch需要RGB)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # image = Image.open(os.path.join('../dataset', self.image_path[idx]))
        if self.transform != None:
            image = self.transform(image=image)['image']
        # 返回结果：测试集只返回图像，训练集返回图像+标签
        if not self.test:
            label = self.label_nums[idx]
            return image, label
        else:
            return image

    def __len__(self):
        '''返回数据集大小（图像数量）'''
        return len(self.image_path)

In [44]:
# 数据增强
transforms_train = albumentations.Compose(
    [
        albumentations.Resize(320, 320),  # 调整图像尺寸到320x320像素
        albumentations.HorizontalFlip(p=0.5),  # 水平翻转（概率50%）
        albumentations.VerticalFlip(p=0.5),  # 垂直翻转（概率50%）
        albumentations.Rotate(limit=180, p=0.7),  # 随机旋转（-180度到+180度，概率70%）
        albumentations.RandomBrightnessContrast(),  # 随机调整亮度和对比度
        
        albumentations.Affine(
            translate_percent=(-0.25, 0.25),   # 平移范围 -25% 到 +25%
            scale=(0.9, 1.1),                  # 缩放范围 90% 到 110%
            rotate=0,                          # 无旋转
            p=0.5                              # 应用概率 50%
        ),
        
        # 图像归一化（使用ImageNet预训练模型的均值和标准差）标准化像素值分布，加速模型收敛
        # 均值: [0.485, 0.456, 0.406]
        # 标准差: [0.229, 0.224, 0.225]
        # max_pixel_value=255.0: 输入图像像素值范围0-255
        albumentations.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225], max_pixel_value=255.0),

        # 将图像从numpy数组转换为PyTorch张量
        # p=1.0: 100%概率应用（总是应用）
        ToTensorV2(p=1.0)
    ]
)

# 不添加随机增强：保证评估结果的一致性
transforms_test = albumentations.Compose(
    [
        albumentations.Resize(320, 320),
        albumentations.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225], max_pixel_value=255.0),
        ToTensorV2(p=1.0)
    ]
)

In [45]:
def train_model(train_loader, valid_loader, test_df, device=torch.device('cuda:0')):
    # 1. 模型初始化
    # 使用预训练的ResNet50模型
    net = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V1)
    # 获取全连接层的输入特征维度
    in_features = net.fc.in_features
    # 替换全连接层以适应176类树叶分类任务
    net.fc = nn.Linear(in_features, 176)
    net = net.to(device)

    # 2. 训练参数设置
    epoch = 30
    best_epoch = 0
    best_score = 0.0
    best_model_state = None  # 保存最佳模型状态
    early_stopping_round = 3  # 早停轮数（验证集性能不再提升时停止）
    losses = []  # 记录每轮训练损失

    # 3. 优化器和损失函数
    optimizer = optim.Adam(net.parameters(), lr=0.0001, weight_decay=1e-5)  # Adam优化器
    loss = nn.CrossEntropyLoss(reduction='mean')  # 交叉熵损失函数
    scheduler = ExponentialLR(optimizer, gamma=0.9)  # 学习率指数衰减调度器

    # 4. 训练循环
    for i in range(epoch):
        acc = 0  # 累计训练准确数
        loss_sum = 0  # 累计训练损失

        # 训练阶段
        net.train()  # 设置模型为训练模式
        for x, y in tqdm(train_loader):  # 遍历训练数据批次
            # 准备输入数据
            x = torch.as_tensor(x, dtype=torch.float).to(device)
            y = y.to(device)

            # 前向传播
            y_hat = net(x)
            # 计算损失
            loss_temp = loss(y_hat, y)
            loss_sum += loss_temp

            # 反向传播和优化
            optimizer.zero_grad()  # 清空梯度
            loss_temp.backward()  # 反向传播计算梯度
            optimizer.step()  # 更新参数

            # 计算准确数
            acc += torch.sum(y_hat.argmax(dim=1).type(y.dtype) == y)
            
        # 更新学习率
        scheduler.step()
        # 记录平均损失
        losses.append(loss_sum.cpu().detach().numpy() / len(train_loader))

        # 打印训练结果
        print("epoch: ", i, 
              "loss=", loss_sum.item(), 
              "train acc: ", (acc/(len(train_loader)*train_loader.batch_size)).item(), end='')

        # 验证阶段
        test_acc = 0  # 累计验证准确数
        net.eval()  # 设置模型为评估模式
        for x, y in tqdm(valid_loader):  # 遍历验证数据批次
            # 准备输入数据
            x = torch.as_tensor(x, dtype=torch.float).to(device)
            y = y.to(device)

            # 前向传播（无梯度计算）
            with torch.no_grad():
                y_hat = net(x)

            # 计算准确数
            test_acc += torch.sum(y_hat.argmax(dim=1).type(y.dtype) == y)

        # 计算并打印验证准确率
        print("valid acc: ", (test_acc / (len(valid_loader)*valid_loader.batch_size)).item())

        # 5. 模型保存与早停
        # 如果当前模型在验证集上表现更好，保存为最佳模型
        if test_acc > best_score:
            best_model_state = copy.deepcopy(net.state_dict())  # 深度复制模型状态
            best_score = test_acc
            best_epoch = i
            print('best epoch save!')

        # 早停机制：如果连续early_stopping_round轮没有提升，停止训练
        if i - best_epoch >= early_stopping_round:
            print(f'Early stopping at epoch {i}')
            break

    # 6. 加载最佳模型
    net.load_state_dict(best_model_state)

    # 7. 在测试集上进行预测
    testset = Leaf_Dataset(test_df, transform=transforms_test, test=True)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, drop_last=False)

    # 进行预测
    predictions = []
    with torch.no_grad():
        for x in tqdm(test_loader):  # 遍历测试数据批次
            x = x.to(device)
            x = torch.as_tensor(x, dtype=torch.float)

            # 前向传播
            y_hat = net(x)

            # 获取预测结果
            predict = torch.argmax(y_hat, dim=1).reshape(-1)
            predict = list(predict.cpu().detach().numpy())
            predictions.extend(predict)
            
    return predictions

In [46]:
# 导入分层K折交叉验证工具
from sklearn.model_selection import StratifiedKFold

# 1. 初始化分层K折交叉验证器
# n_splits=5: 将数据分为5折
# shuffle=True: 打乱数据顺序
# random_state=2025: 随机种子（保证每次划分结果相同）
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2025)

# 2. 创建空 DataFrame 存储各折的预测结果
prediction_df = pd.DataFrame()

# 3. 开始K折交叉验证循环
# enumerate(skf.split(train, train['number'])):
#   - train: 整个训练集DataFrame
#   - train['number']: 数字标签（用于分层抽样）
for fold_n, (train_idx, val_idx) in enumerate(skf.split(train, train['number'])):
    print(f'fold {fold_n} training...')  # 打印当前折信息

    # 4. 划分训练集和验证集
    # 根据索引获取当前折的训练数据和验证数据
    train_data = train.iloc[train_idx]
    eval_data = train.iloc[val_idx]

    # 5. 创建数据集对象
    # 训练集应用训练时的数据增强
    trainset = Leaf_Dataset(train_data, transform=transforms_train)
    # 验证集应用测试时的变换（无增强）
    evalset = Leaf_Dataset(eval_data, transform=transforms_test)

    # 6. 创建数据加载器
    # 训练集 DataLoader：打乱顺序，用于训练
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, drop_last=False)
    eval_loader = torch.utils.data.DataLoader(evalset, batch_size=32, shuffle=False, drop_last=False)

    # 7. 训练模型并在测试集上预测
    predictions = train_model(train_loader, eval_loader, test_df)

    # 8. 存储当前折的预测结果
    prediction_df[f'fold_{fold_n}'] = predictions

fold 0 training...


  0%|          | 0/459 [00:00<?, ?it/s]

epoch:  0 loss= 1290.000244140625 train acc:  0.3598856031894684

  0%|          | 0/115 [00:00<?, ?it/s]

valid acc:  0.5986412763595581
best epoch save!


...

  0%|          | 0/459 [00:00<?, ?it/s]

epoch:  20 loss= 24.2436466217041 train acc:  0.9826388359069824

  0%|          | 0/115 [00:00<?, ?it/s]

valid acc:  0.9671195149421692
Early stopping at epoch 20


  0%|          | 0/138 [00:00<?, ?it/s]

In [47]:
print(prediction_df)

      fold_0  fold_1  fold_2  fold_3  fold_4
0         37      37      37      37      37
1        159     159     159     159     159
2        133     133     133     133     133
3        117     117     117     117     117
4        133     133     133     133     133
...      ...     ...     ...     ...     ...
8795     168     168     168     168     116
8796      59      59      59      59      59
8797     168     168     168     168     168
8798     121     121     121     121     121
8799     121     121     121     121     121

[8800 rows x 5 columns]


In [48]:
# 1. 获取各交叉验证折的最终预测结果（众数投票）
# prediction_df包含各折的预测结果（每列代表一折的预测）
# mode(axis=1)[0]：对每行取众数, 当存在多个众数时，取第一个出现的众数
# astype(int)：将结果转换为整数类型（类别索引）
all_predictions = list(prediction_df.mode(axis=1)[0].astype(int))

# 2. 将数字标签转换回原始文本标签
predict_label = []
for i in range(len(all_predictions)):
    # 使用之前创建的 labels_unique 列表（包含所有唯一类别名称）
    # 将数字索引转换为对应的类别名称
    predict_label.append(labels_unique[all_predictions[i]])

# 3. 准备提交文件
# 读取测试集原始CSV（包含image列）
submission = pd.read_csv('../dataset/test.csv')

# 4. 添加预测结果列
# 将预测的文本标签添加到 DataFrame 的新列'label'中
submission['label'] = predict_label

# 5. 保存结果文件
# 不保存索引列（index=False）
submission.to_csv('../dataset/result.csv', index=False)

In [49]:
submission

,image,label
0,images/18353.jpg,asimina_triloba
1,images/18354.jpg,betula_nigra
2,images/18355.jpg,platanus_acerifolia
3,images/18356.jpg,pinus_bungeana
4,images/18357.jpg,platanus_acerifolia
...,...,...
8795,images/27148.jpg,pinus_thunbergii
8796,images/27149.jpg,crataegus_crus-galli
8797,images/27150.jpg,pinus_thunbergii
8798,images/27151.jpg,juniperus_virginiana
